<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">2018-2 · Máster universitario en Ciencia de datos (Data Science)</p>
<p style="margin: 0; text-align:right;">TFM · Sistema Reactivo Sobre Dispositivos IoT</p>
</div>
</div>
<div style="width:0%;">&nbsp;</div>


# TFM: Sistema Reactivo Sobre Dispositivos IoT - Parte III

Este notebook contiene la parte de generación del modelo del trabajo realizado para el TFM.

In [1]:
# carga de librerías necesarias:
%matplotlib inline
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import norm
from math import ceil
from math import floor
import warnings
warnings.filterwarnings('ignore')

#imports para generar el modelo
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

c:\users\ricky\appdata\local\programs\python\python36\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
#Se carga el fichero con el conjunto de datos:
dataset = pd.read_excel("./datos/dataset_fecha_hora_clean.xlsx",
                        header=[0],
                        columns = ['HORA','HORA_MIN','HORA_NORM','TIPO_USER','TIPO_USER_INT','ID_ACCION','TIPO_DIA','TIPO_DIA_INT'])

#se carga el conjunto con la descripción de las acciones: 
dataset_acciones = pd.read_excel("./datos/dataset_acciones.xlsx",
                                 header=[0],
                                 columns = ['ID_ACCION','ACCION'])

In [3]:
#tras las pruebas anteriores, modificamos el intervalo normalizado a 5 minutos
INTERVALO = 5
dataset['HORA_NORM'] = [ceil(hora_min/INTERVALO) for hora_min in dataset['HORA_MIN']]

In [4]:
#Se seleccionan las columnas con las que se va a trabajar:
cols_trabajo = ['HORA_NORM','TIPO_USER_INT','ID_ACCION','TIPO_DIA_INT']
predictor_cols = ['TIPO_USER_INT','ID_ACCION','TIPO_DIA_INT']
target_col = ['HORA_NORM']
dataset = dataset[cols_trabajo]

In [5]:
#almacenamos las columnas de datos y predicción en dos arrays
X = np.array(dataset[predictor_cols])
y = np.array(dataset[target_col])

#dividimos los datos en conjunto de entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1028)

In [6]:
# se define el clasificador RandomForestClassifier
clf = RandomForestClassifier(n_jobs=1, random_state=0, max_depth=10)

# se entrena el modelo y se generan las predicciones
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

##########Cálculo de exactitud##########
acc_score = accuracy_score(y_test, y_pred, normalize=False)
print('Precisión de la predicción: {}%'.format(round(clf.score(X_test,y_test)*100,2)))
print('Predicciones erróneas:',len(y_test)-acc_score,'/',len(y_test))

Precisión de la predicción: 64.22%
Predicciones erróneas: 2181 / 6096


In [7]:
#analizamos para cada tipo de usuario, utilizando únicamente RandomForest
for tipo_user in dataset['TIPO_USER_INT'].unique():

    dataset_aux = dataset.loc[dataset['TIPO_USER_INT'] == tipo_user][cols_trabajo]
    
    X = np.array(dataset_aux[predictor_cols])
    y = np.array(dataset_aux[target_col])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1028)
    
    clf.fit(X_train, y_train)
    print('Usuario {}:\tPrecisión de la predicción: {}%'.format(str(tipo_user),round(clf.score(X_test,y_test)*100,2)))

Usuario 1:	Precisión de la predicción: 75.68%
Usuario 2:	Precisión de la predicción: 73.47%
Usuario 3:	Precisión de la predicción: 45.44%


In [8]:
#analizamos para cada acción, utilizando únicamente RandomForest
for id_accion in dataset['ID_ACCION'].unique():
    
    dataset_aux = dataset.loc[dataset['ID_ACCION'] == id_accion][cols_trabajo]
    
    X = np.array(dataset_aux[predictor_cols])
    y = np.array(dataset_aux[target_col])
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1028)

    clf.fit(X_train, y_train)
    print('Acción {}:\tPrecisión de la predicción: {}%'.format(str(id_accion),round(clf.score(X_test,y_test)*100,2)))

Acción 28:	Precisión de la predicción: 76.67%
Acción 29:	Precisión de la predicción: 83.22%
Acción 20:	Precisión de la predicción: 43.62%
Acción 11:	Precisión de la predicción: 74.64%
Acción 3:	Precisión de la predicción: 60.21%
Acción 19:	Precisión de la predicción: 49.85%
Acción 9:	Precisión de la predicción: 87.32%
Acción 27:	Precisión de la predicción: 85.37%
Acción 5:	Precisión de la predicción: 46.7%
Acción 10:	Precisión de la predicción: 85.61%
Acción 4:	Precisión de la predicción: 78.42%
Acción 22:	Precisión de la predicción: 70.5%
Acción 15:	Precisión de la predicción: 24.26%
Acción 24:	Precisión de la predicción: 65.67%
Acción 23:	Precisión de la predicción: 55.56%
Acción 21:	Precisión de la predicción: 58.96%
Acción 6:	Precisión de la predicción: 51.85%
Acción 8:	Precisión de la predicción: 62.5%
Acción 2:	Precisión de la predicción: 55.07%
Acción 12:	Precisión de la predicción: 90.2%
Acción 7:	Precisión de la predicción: 70.71%
Acción 13:	Precisión de la predicción: 88.24%


In [9]:
#se analiza para cada usuario, tipo de día y acción, utilizando únicamente RandomForest

ac_validas = [] #en esta lista se almacenarán las acciones que son válidas para proponer automatizar

for tipo_user in dataset['TIPO_USER_INT'].unique():
    print('\n------------Análisis para el usuario: {}------------'.format(str(tipo_user)))
    
    dataset_user = dataset.loc[dataset['TIPO_USER_INT'] == tipo_user][cols_trabajo]
    
    for tipo_dia in dataset_user['TIPO_DIA_INT'].unique():
        
        if tipo_dia == 0:
            print('-Días de entre semana:')
        else:
            if tipo_dia == 1:
                print('-Días de fin de semana:')
        
        dataset_tipo_dia = dataset_user.loc[dataset_user['TIPO_DIA_INT'] == tipo_dia][cols_trabajo]
        
        for id_accion in dataset_user['ID_ACCION'].unique():

            dataset_accion = dataset_tipo_dia.loc[dataset_tipo_dia['ID_ACCION'] == id_accion][cols_trabajo]

            X = np.array(dataset_accion[predictor_cols])
            y = np.array(dataset_accion[target_col])
            
            if (X.shape[0]) > 0:
                
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1028)

                clf.fit(X_train, y_train)

                score = round(clf.score(X_test,y_test),2)
                
                aux_pred = np.mean(clf.predict(X_test))
                time_norm = aux_pred*INTERVALO
                horas = int(time_norm//60)
                minutos = int(time_norm%60)
                time = '{:>02d}'.format(horas)+':'+'{:<02d}'.format(minutos)
                    
                print('Acción {}:\tPrecisión de la predicción: {}% - hora: {}'.format(str(id_accion),round(score*100,2),time))

                #se establece un acierto mínimo para la acciones "válidas" para automatizar
                ACIERTO_MINIMO = 0.80
                if score >= ACIERTO_MINIMO:
                    accion = dataset_acciones.loc[dataset_acciones['ID_ACCION'] == id_accion][['ACCION']].ACCION.item()

                    ac_validas.append([tipo_user,tipo_dia,id_accion,accion,score,time_norm,time])


------------Análisis para el usuario: 1------------
-Días de fin de semana:
Acción 28:	Precisión de la predicción: 85.0% - hora: 08:35
Acción 29:	Precisión de la predicción: 85.0% - hora: 09:50
Acción 20:	Precisión de la predicción: 60.0% - hora: 09:50
Acción 11:	Precisión de la predicción: 86.0% - hora: 09:20
Acción 3:	Precisión de la predicción: 88.0% - hora: 11:50
Acción 19:	Precisión de la predicción: 78.0% - hora: 20:20
Acción 9:	Precisión de la predicción: 83.0% - hora: 21:50
Acción 27:	Precisión de la predicción: 85.0% - hora: 22:35
Acción 5:	Precisión de la predicción: 80.0% - hora: 23:35
-Días de entre semana:
Acción 28:	Precisión de la predicción: 83.0% - hora: 06:50
Acción 29:	Precisión de la predicción: 86.0% - hora: 07:10
Acción 20:	Precisión de la predicción: 41.0% - hora: 06:55
Acción 11:	Precisión de la predicción: 100.0% - hora: 07:10
Acción 3:	Precisión de la predicción: 40.0% - hora: 20:20
Acción 19:	Precisión de la predicción: 76.0% - hora: 20:20
Acción 9:	Precisió

In [10]:
df_ac_validas = pd.DataFrame(np.array(ac_validas).reshape(len(ac_validas),7), columns = ['TIPO_USER','TIPO_DIA','ID_ACCION','ACCION','SCORE','HORA_NORM','HORA'])

#se define un diccionario para decodificar las columnas TIPO_USER y TIPO_DIA
convers_int = {"TIPO_USER": {"1":"USER_1", "2":"USER_2", "3":"USER_3"},
               "TIPO_DIA": {"0":"SEM", "1":"WKN"}}

df_ac_validas.replace(convers_int, inplace=True)

#se almacena el dataset ordenado
df_ac_validas = df_ac_validas[['TIPO_USER','TIPO_DIA','ID_ACCION','ACCION','SCORE','HORA']]
df_ac_validas = df_ac_validas.sort_values(by=['TIPO_USER','TIPO_DIA','SCORE'], ascending=[True,True,False]).reset_index(drop=True)

#se muestran las acciones que podrían ser candidatas a automatizarse:
df_ac_validas.head(50)

,TIPO_USER,TIPO_DIA,ID_ACCION,ACCION,SCORE,HORA
0,USER_1,SEM,11,Dame el parte de tiempo,1.0,07:10
1,USER_1,SEM,12,Dame el parte de tráfico,0.9,07:10
2,USER_1,SEM,7,Apaga todas las luces,0.88,07:35
3,USER_1,SEM,13,Enciende la calefacción,0.88,17:35
4,USER_1,SEM,1,Apaga la calefacción,0.87,22:35
5,USER_1,SEM,29,Sube la persiana del salón,0.86,07:10
6,USER_1,SEM,9,Baja todas las persianas,0.86,20:35
7,USER_1,SEM,25,Programa el despertador para las 6:40,0.86,22:35
8,USER_1,SEM,28,Sube la persiana de la habitación,0.83,06:50
9,USER_1,SEM,5,Apaga la televisión,0.82,22:15


In [11]:
#se exportan a un fichero excel las acciones "automatizables"
df_ac_validas.to_excel("./datos/dataset_acciones_automatizables.xlsx",
                  columns = ['TIPO_USER','TIPO_DIA','ID_ACCION','ACCION','SCORE','HORA'],
                  engine='xlsxwriter')